In [38]:
import pandas as pd
import torch
import mysql.connector
import dgl.data
import numpy as np
from dgl import save_graphs, load_graphs
import torch as th
import gensim
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import requests
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\david/.cache\huggingface\hub\models--allenai--scibert_scivocab_uncased\snapshots\24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1\config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
 

loading weights file pytorch_model.bin from cache at C:\Users\david/.cache\huggingface\hub\models--allenai--scibert_scivocab_uncased\snapshots\24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1\pytorch_model.bin
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (in

In [39]:
input_text = "This is an example sentence."
tokenized_input = tokenizer.encode_plus(input_text, return_tensors='pt')

In [42]:
outputs = model(**tokenized_input)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4281,  1.1507,  1.0390,  ...,  0.5818, -0.7117,  0.4247],
         [-0.0830,  0.4637, -0.3874,  ...,  0.2196,  0.0861,  0.1827],
         [ 0.9171,  0.1888, -0.6332,  ..., -1.0889,  0.5068,  0.2355],
         ...,
         [ 1.1826,  0.8551,  0.9354,  ...,  0.7910, -0.6270,  0.3596],
         [ 1.2706,  1.2989,  1.0110,  ..., -0.5780, -1.8531,  0.3310],
         [-0.3739,  0.8233,  0.6964,  ...,  0.2347, -0.6886,  0.3500]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.2122, -0.1372,  0.8959,  0.9986,  0.2182,  0.9992,  0.2944, -1.0000,
          0.3152, -0.8880,  0.0150,  0.1978,  0.2651,  0.9643, -0.0364, -0.0407,
         -0.2893, -0.1392, -0.9857, -0.9939, -0.0296,  0.2299,  0.5828,  0.1879,
         -0.1929,  0.0944, -0.9982,  0.0736, -0.1445, -0.0243,  0.1224, -0.8223,
         -0.1183, -0.3029,  0.5714,  0.2225,  0.9997, -0.1818,  0.2227,  0.9620,
         -0.3864,  0.3054, -0.02

# Connect to database

In [2]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [3]:
test_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_papers.append(x)
in_params_test = ','.join(['%s'] * len(test_papers))

train_papers = []
with open(r'./train_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        train_papers.append(x)

total_papers = train_papers + test_papers
in_params_train = ','.join(['%s'] * len(train_papers))

val_papers = []
with open(r'./val_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        val_papers.append(x)

in_params_val = ','.join(['%s'] * len(val_papers))
in_params = ','.join(['%s'] * len(train_papers + test_papers))

## Fetch Data primary

In [4]:
def remap(x, mapping):
    return mapping.get(x, x)

In [5]:
cursor.execute("select b.ReferenceID, b.ReferencedByID from referencedBy b, Papers p where b.ReferencedByID in (%s) and b.ReferenceID = p.PaperID and p.`primary author` in (select AuthorID from affiliatedTo) and b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)" % in_params, total_papers)
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)

remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

all_papers_list = all_papers.tolist()
all_params = ','.join(['%s'] * len(all_papers_list))

cursor.execute("select p.PaperID, p.`primary author`, a.Gender, p.Title from Papers p, Authors a where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = a.AuthorID" % all_params, all_papers_list)
author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

## Create features for each node

In [6]:
model_path = './GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [7]:
def title_to_vec(title):
    title = title.replace("-", " ")
    tokens = nltk.word_tokenize(title)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    vectors = []
    for word in tokens:
        try:
            vec = model[word.lower()]
            vectors.append(vec)
        except KeyError:
            #print(word)
            continue
    title_vector = None
    try:
        title_vector = sum(vectors) / len(vectors)
    except ZeroDivisionError:
        pass
    return title_vector

In [8]:
all_papers_info = pd.DataFrame(author_edges)
all_papers_info[0] = all_papers_info[0].astype(int)
all_papers_info = all_papers_info.sort_values(0)
paper_feats = []
stop_words = set(stopwords.words('english'))
for title in all_papers_info[3]:
    title_vec = title_to_vec(title)
    paper_feats.append(title_vec)
paper_feats = torch.tensor(paper_feats)

C:\Users\david\AppData\Local\Temp\ipykernel_3248\1550066203.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  paper_feats = torch.tensor(paper_feats)


In [9]:
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [10]:
def request_author_information(authors):
    resp = requests.post("https://api.semanticscholar.org/graph/v1/author/batch?fields=papers.title", headers=headers, json={"ids": authors}).json()
    return resp

def request_author_individual_dfinformation(authors):
    resp = requests.post("https://api.semanticscholar.org/graph/v1/author/batch?fields=papers.title", headers=headers, json={"ids": authors}).json()
    return resp

In [11]:
import time
index = 0
start = 100
author_feats = []
while start*index <= len(all_authors):
    print(f"{start*index}/{len(all_authors)}")
    batch_authors = all_authors[start*index: (start*index)+start]
    index += 1
    while True:
        response_batch = request_author_information(batch_authors.tolist())
        if "message" not in response_batch:
            break
        time.sleep(2)
    print(len(response_batch))
    #print(response_batch)
    for response in response_batch:
        if not response:
            author_hist = np.zeros(300)
            author_feats.append(author_hist)
            continue

        author_papers = response["papers"]
        #print(author_papers)
        author_hist = []
        for paper in author_papers:
            paperID = paper["paperId"]
            if paperID in test_papers or paperID in train_papers:
                #paperID in train_papers or
                continue
            title = paper["title"]
            title_vect = title_to_vec(title)
            if title_vect is not None:
                author_hist.append(title_vect)
        if len(author_hist) != 0:
            author_hist = np.mean(np.array(author_hist), axis=0)
        else:
            author_hist = np.zeros(300)
            print("zero_vector")
        author_feats.append(author_hist)

0/18318
100
100/18318
100
200/18318
100
300/18318
100
400/18318
100
500/18318
100
600/18318
100
700/18318
100
800/18318
100
zero_vector
900/18318
100
1000/18318
100
zero_vector
zero_vector
1100/18318
100
zero_vector
zero_vector
1200/18318
100
zero_vector
zero_vector
1300/18318
100
zero_vector
zero_vector
1400/18318
100
zero_vector
1500/18318
100
1600/18318
100
1700/18318
100
zero_vector
1800/18318
100
1900/18318
100
2000/18318
100
zero_vector
2100/18318
100
2200/18318
100
2300/18318
100
2400/18318
100
2500/18318
100
zero_vector
2600/18318
100
zero_vector
2700/18318
100
zero_vector
2800/18318
100
2900/18318
100
3000/18318
100
3100/18318
100
3200/18318
100
3300/18318
100
3400/18318
100
3500/18318
100
3600/18318
100
zero_vector
zero_vector
zero_vector
zero_vector
zero_vector
3700/18318
100
zero_vector
zero_vector
zero_vector
3800/18318
100
zero_vector
3900/18318
100
zero_vector
4000/18318
100
zero_vector
4100/18318
100
zero_vector
4200/18318
100
4300/18318
100
4400/18318
100
4500/18318
10

In [12]:
author_feats_vec = torch.tensor(author_feats)

In [13]:
author_feats_vec.shape

torch.Size([18318, 300])

In [ ]:
link = f"https://api.semanticscholar.org/graph/v1/paper/search?query={text}"
response = requests.get(link, headers=headers).json()

In [14]:
cursor.execute("select distinct p.`primary author` from Papers p where p.PaperID in (%s)" % in_params, total_papers)
authors_in_question = np.array(cursor.fetchall()).flatten()

## Fetch the Gender

In [15]:
all_params_gender = ','.join(['%s'] * len(all_authors))

In [16]:
cursor.execute("select distinct AuthorID, Gender from Authors a where a.AuthorID in (%s)" % all_params_gender, all_authors.tolist())
gender_edges = np.array(cursor.fetchall())

In [17]:
gender_edges[:,0] = vec_remap(gender_edges[:,0], remapped_authors)

## Fetch the affiliation

In [18]:
cursor.execute("select p.PaperID, p.`primary author`, b.AffiliationID, b.Country from Papers p, affiliatedTo af, Affiliations b where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = af.AuthorID and af.affiliatedTo = b.AffiliationID" % all_params, all_papers_list)
author_info = np.array(cursor.fetchall())

all_affiliations = np.unique(author_info[:,2])
remapped_affiliations = {all_affiliations[i]: i  for i in range(len(all_affiliations))}

all_countries = np.unique(author_info[:,3])
remapped_countries = {all_countries[i]: i  for i in range(len(all_countries))}

author_info[:,2] = vec_remap(author_info[:,2], remapped_affiliations)
author_info[:,3] = vec_remap(author_info[:,3], remapped_countries)
author_info[:,1] = vec_remap(author_info[:,1], remapped_authors)
author_info[:,0] = vec_remap(author_info[:,0], remapped_papers)

affiliation_edges = author_info[:,[2,1]]
affiliation_edges = np.unique(affiliation_edges, axis=0)

country_edges = author_info[:,[3,2]]
country_edges = np.unique(country_edges, axis=0)

# Fetch data

In [5]:
cursor.execute("select * from referencedBy b where b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)")
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)
remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

cursor.execute("select * from authoredBy where PaperID in (select PaperID from Papers) and (PaperID in (select ReferenceID from referencedBy) or PaperID in (select ReferencedByID from referencedBy)) and PaperID in (select p.PaperID from Papers p where p.Leaf = FALSE) and AuthoredByID in (select a.AuthorID from affiliatedTo a)")

author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [21]:
cursor.execute("select distinct b.AuthoredByID from Papers p, authoredBy b where p.Leaf = False and p.PaperID in (%s) and p.PaperID = b.PaperID and b.AuthoredByID in (select a.AuthorID from affiliatedTo a)" % in_params, train_papers+test_papers)
authors_in_question = np.array(cursor.fetchall()).flatten()

# Paper data

Since we use only paper information, we remove all other "authoredBy" edges

In [19]:
remapped_test_papers = th.tensor([remapped_papers[idx] for idx in test_papers])
remapped_val_papers = th.tensor([remapped_papers[idx] for idx in val_papers])
remapped_train_papers = th.tensor([remapped_papers[idx] for idx in train_papers])
remapped_authors_in_question = th.tensor([remapped_authors[idx] for idx in authors_in_question])

In [20]:
author_feats_vec

tensor([[-0.0146,  0.0525, -0.0199,  ..., -0.0717,  0.0053, -0.0091],
        [-0.0028,  0.0341,  0.0362,  ..., -0.0184,  0.0457, -0.0389],
        [-0.0012, -0.0004,  0.0246,  ..., -0.0462, -0.0197,  0.0109],
        ...,
        [ 0.0262,  0.0078, -0.0416,  ..., -0.0869, -0.0445, -0.0357],
        [-0.0312,  0.0735,  0.0188,  ..., -0.0209,  0.0594,  0.0109],
        [ 0.0610,  0.0380, -0.0386,  ..., -0.0682, -0.0064, -0.0540]],
       dtype=torch.float64)

In [285]:
tensors_in_question = torch.index_select(author_feats_vec,0,remapped_authors_in_question)

In [288]:
only_zeros = [idx for idx, tensor in enumerate(author_feats_vec) if (tensor == 0).all()]

In [290]:
len(only_zeros)

42

In [277]:
remapped_authors_in_question[442]

tensor(6511)

In [279]:
list(remapped_authors)[6511]

'2062795769'

In [21]:
hetero_graph = dgl.heterograph(
    {
        ('paper', 'authored', 'author'): (author_edges[:,0].astype(int), author_edges[:,1].astype(int)),
        ('author', "writes", "paper"): (author_edges[:,1].astype(int), author_edges[:,0].astype(int)),
        ('paper', 'cites', 'paper'): (paper_edges[:,0].astype(int), paper_edges[:,1].astype(int)),
        ('gender', 'gendered', 'author'): (gender_edges[:,1].astype(int), gender_edges[:,0].astype(int)),
        ('paper', 'authored_gender', 'gender'): (author_edges[:,0].astype(int), author_edges[:,2].astype(int)),
        ('paper', 'authored_affiliation', 'affiliation'): (author_info[:,0].astype(int), author_info[:,2].astype(int)),
        ('affiliation', 'affiliated', 'author'): (affiliation_edges[:,0].astype(int), affiliation_edges[:,1].astype(int)),
        ('paper', 'authored_country', 'country'): (author_info[:,0].astype(int), author_info[:,3].astype(int)),
        ('country', 'contains', 'affiliation'): (country_edges[:,0].astype(int), country_edges[:,1].astype(int))
    }
)
hetero_graph.nodes['author'].data['feature'] = author_feats_vec
hetero_graph.nodes['paper'].data['feature'] = paper_feats

In [22]:
"""
edges = hetero_graph.edges(etype="authored")
author_mask = ~torch.isin(edges[1], remapped_authors_in_question)
paper_edges = edges[0][author_mask]
author_edges = edges[1][author_mask]
author_ids_remove = hetero_graph.edge_ids(paper_edges, author_edges, etype="authored")
hetero_graph = dgl.remove_edges(hetero_graph, author_ids_remove, etype="authored")
"""

'\nedges = hetero_graph.edges(etype="authored")\nauthor_mask = ~torch.isin(edges[1], remapped_authors_in_question)\npaper_edges = edges[0][author_mask]\nauthor_edges = edges[1][author_mask]\nauthor_ids_remove = hetero_graph.edge_ids(paper_edges, author_edges, etype="authored")\nhetero_graph = dgl.remove_edges(hetero_graph, author_ids_remove, etype="authored")\n'

### Split into Training, Validation, Test set

#### Train set

In [23]:
edges = hetero_graph.edges(etype="authored")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = hetero_graph.edge_ids(train_src, train_dst, etype="authored")
train_hetero_graph = dgl.remove_edges(hetero_graph, train_ids, etype="authored")

train_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 2384, ('paper', 'authored_affiliation', 'affiliation'): 36285, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 36099, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [24]:
edges = train_hetero_graph.edges(etype="authored_gender")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_gender")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_gender")

train_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 2384, ('paper', 'authored_affiliation', 'affiliation'): 36285, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 2384, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [25]:
edges = train_hetero_graph.edges(etype="authored_affiliation")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_affiliation")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_affiliation")

train_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 2384, ('paper', 'authored_affiliation', 'affiliation'): 2384, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 2384, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [26]:
edges = train_hetero_graph.edges(etype="authored_country")

train_mask = ~torch.isin(edges[0], remapped_train_papers)
train_src = edges[0][train_mask]
train_dst = edges[1][train_mask]

train_ids = train_hetero_graph.edge_ids(train_src, train_dst, etype="authored_country")
train_hetero_graph = dgl.remove_edges(train_hetero_graph, train_ids, etype="authored_country")

train_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 2384, ('paper', 'authored_affiliation', 'affiliation'): 2384, ('paper', 'authored_country', 'country'): 2467, ('paper', 'authored_gender', 'gender'): 2384, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

# Validation Set

In [27]:
edges = hetero_graph.edges(etype="authored")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = hetero_graph.edge_ids(val_src, val_dst, etype="authored")
val_hetero_graph = dgl.remove_edges(hetero_graph, val_ids, etype="authored")
val_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 36285, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 36099, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [28]:
edges = val_hetero_graph.edges(etype="authored_gender")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_gender")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_gender")
val_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 36285, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [29]:
edges = val_hetero_graph.edges(etype="authored_affiliation")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_affiliation")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_affiliation")
val_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 300, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [30]:
edges = val_hetero_graph.edges(etype="authored_country")
val_mask = ~torch.isin(edges[0], remapped_val_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = val_hetero_graph.edge_ids(val_src, val_dst, etype="authored_country")
val_hetero_graph = dgl.remove_edges(val_hetero_graph, val_ids, etype="authored_country")
val_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 300, ('paper', 'authored_affiliation', 'affiliation'): 300, ('paper', 'authored_country', 'country'): 383, ('paper', 'authored_gender', 'gender'): 300, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

## Test set

In [31]:
edges = hetero_graph.edges(etype="authored")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = hetero_graph.edge_ids(val_src, val_dst, etype="authored")
test_hetero_graph = dgl.remove_edges(hetero_graph, val_ids, etype="authored")
test_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 1005, ('paper', 'authored_affiliation', 'affiliation'): 36285, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 36099, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [32]:
edges = test_hetero_graph.edges(etype="authored_gender")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_gender")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_gender")
test_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 1005, ('paper', 'authored_affiliation', 'affiliation'): 36285, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 1005, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [33]:
edges = test_hetero_graph.edges(etype="authored_affiliation")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_affiliation")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_affiliation")
test_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 1005, ('paper', 'authored_affiliation', 'affiliation'): 1005, ('paper', 'authored_country', 'country'): 36285, ('paper', 'authored_gender', 'gender'): 1005, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [34]:
edges = test_hetero_graph.edges(etype="authored_country")
val_mask = ~torch.isin(edges[0], remapped_test_papers)
val_src = edges[0][val_mask]
val_dst = edges[1][val_mask]

val_ids = test_hetero_graph.edge_ids(val_src, val_dst, etype="authored_country")
test_hetero_graph = dgl.remove_edges(test_hetero_graph, val_ids, etype="authored_country")
test_hetero_graph

Graph(num_nodes={'affiliation': 2342, 'author': 18318, 'country': 88, 'gender': 2, 'paper': 36099},
      num_edges={('affiliation', 'affiliated', 'author'): 18422, ('author', 'writes', 'paper'): 36099, ('country', 'contains', 'affiliation'): 2342, ('gender', 'gendered', 'author'): 18318, ('paper', 'authored', 'author'): 1005, ('paper', 'authored_affiliation', 'affiliation'): 1005, ('paper', 'authored_country', 'country'): 1088, ('paper', 'authored_gender', 'gender'): 1005, ('paper', 'cites', 'paper'): 138804},
      metagraph=[('affiliation', 'author', 'affiliated'), ('author', 'paper', 'writes'), ('paper', 'author', 'authored'), ('paper', 'affiliation', 'authored_affiliation'), ('paper', 'country', 'authored_country'), ('paper', 'gender', 'authored_gender'), ('paper', 'paper', 'cites'), ('country', 'affiliation', 'contains'), ('gender', 'author', 'gendered')])

In [35]:
save_graphs("./graphs/hetero_graphs_primary_word2vec_w_zeros.bin", [train_hetero_graph, val_hetero_graph, test_hetero_graph])